# Voting Classifier

In [1]:
import pandas as pd
import glob
import os
import mlflow

from sklearn.preprocessing import MinMaxScaler

In [4]:
# load all results from the experiment
df = mlflow.search_runs(experiment_names=["Titanic"])

df_best = df[df["run_id"].isin([
    "5714c08f245d4b11b7e60fdb97fe5b3e",
    "37ba72127e6d4572bf74163456c00d91",
    "0f8ca2d31e2643f6ac443ecc38403150",
    "c878a11f9e964567aafea50532c590b7"
])][["run_id", "metrics.best_cv_score", "metrics.kaggle_score"]]

df_best["score_scaled"] = MinMaxScaler(feature_range=(0.1,0.9)).fit_transform(df_best["metrics.kaggle_score"].values.reshape(len(df_best), -1))

df_best.head(10)

,run_id,metrics.best_cv_score,metrics.kaggle_score,score_scaled
101,5714c08f245d4b11b7e60fdb97fe5b3e,0.841,0.77751,0.100000
254,c878a11f9e964567aafea50532c590b7,0.834,0.78229,0.499582
305,0f8ca2d31e2643f6ac443ecc38403150,0.827,0.78229,0.499582
509,37ba72127e6d4572bf74163456c00d91,0.846,0.78708,0.900000


In [5]:
# load the submissions of the best results
path = r'E:\Data Science Projects\Kaggle\titanic\04_MachineLearningModels\submissions'
li = []

for submission in df_best.run_id:
    submission_path = path + "\\" + submission + ".csv"
    df = pd.read_csv(submission_path, index_col="PassengerId", header=0)
    df.rename(columns={"Survived": submission}, inplace=True)
    li.append(df)

df_voting = pd.concat(li, axis=1, ignore_index=False).sort_values(by="PassengerId")

df_voting.head()

,5714c08f245d4b11b7e60fdb97fe5b3e,c878a11f9e964567aafea50532c590b7,0f8ca2d31e2643f6ac443ecc38403150,37ba72127e6d4572bf74163456c00d91
PassengerId,,,,
892,0,0,0,0
893,0,1,0,1
894,0,0,0,0
895,0,0,0,0
896,1,1,1,1


In [6]:
# stack the results by the run_id so that all runs are in one column
df_voting_stack = df_voting.stack().reset_index().rename(columns={"level_1": "run_id", 0: "survived"})
df_voting_stack.head()

,PassengerId,run_id,survived
0,892,5714c08f245d4b11b7e60fdb97fe5b3e,0
1,892,c878a11f9e964567aafea50532c590b7,0
2,892,0f8ca2d31e2643f6ac443ecc38403150,0
3,892,37ba72127e6d4572bf74163456c00d91,0
4,893,5714c08f245d4b11b7e60fdb97fe5b3e,0


In [7]:
# merge the score_scaled to each run_id 
df_voting_stack = df_voting_stack.merge(df_best[["run_id", "score_scaled"]], on="run_id")
df_voting_stack.head()

,PassengerId,run_id,survived,score_scaled
0,892,5714c08f245d4b11b7e60fdb97fe5b3e,0,0.1
1,893,5714c08f245d4b11b7e60fdb97fe5b3e,0,0.1
2,894,5714c08f245d4b11b7e60fdb97fe5b3e,0,0.1
3,895,5714c08f245d4b11b7e60fdb97fe5b3e,0,0.1
4,896,5714c08f245d4b11b7e60fdb97fe5b3e,1,0.1


In [8]:
# create a pivot table for each of the binary results and sum the score_scaled by each of the binary results
df_voting_stack = pd.pivot_table(data=df_voting_stack, values="score_scaled", index="PassengerId", columns="survived", aggfunc="sum")
df_voting_stack.head()

survived,0,1
PassengerId,,
892,1.999164,NaN
893,0.599582,1.399582
894,1.999164,NaN
895,1.999164,NaN
896,NaN,1.999164


In [9]:
# use the idmax function to get the column name for the row where the cell value is the greatest 
df_submission = pd.read_csv("../01_RawData/gender_submission.csv")
df_submission['Survived'] = df_voting_stack.idxmax(axis=1).reset_index()[0].astype(int)

df_submission.to_csv('submissions/voting.csv', index=False)
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


The final results with the score_scaled feature 0.79186 is higher compared the best individual score with 0.78229.
This voting classifier results in a position of the leaderbord: 820...1051 of around 14k